# Why..🤔
<img src="https://velog.velcdn.com/images/bailando/post/d492013a-ccf0-4ddc-acae-2d842622c3fd/image.png" width="500" height="250">
<img src="https://velog.velcdn.com/images/bailando/post/3b24d368-05bd-438a-bddd-0ecf8d44763e/image.jpeg" width="500" height="1000">

위와 같은 문제를 만났을 때 하나의 방향을 더 생각해 볼 수 있다~~


<!-- 4. 5 Handling Text Files ~~ 4. 6 Normalizing Unicode for Reliable Comparisons -->
## 4. 5 텍스트 파일다루기
텍스트를 처리하는 최고의 방법은 '유니코드 샌드위치'다.
가능하면 bytes를 str로 변환해야 한다는 것을 의미한다.
![](https://www.oreilly.com/api/v2/epubs/9781492056348/files/assets/flpy_0402.png)

가능하면 빨리 bytes를 str으로 변환하고 가능한 늦게 str을 bytes로 변환한다. 

파이썬 3에서는 내장된 `open()` 함수를 통해 파일을 텍스트 모드로 읽고 쓸 때 필요한 인코딩과 디코딩 작업을 모두 수행해준다. 
따라서 file.read()에서 str객체를 가져와 처리하고 file.write()에 전달하면 된다.


- reference
    - [ASCII & Unicode](https://whatisthenext.tistory.com/103)
    - [한글 인코딩의 이해](https://d2.naver.com/helloworld/19187)

In [1]:
open('cafe.txt', 'w', encoding='utf_8').write('café')       # cafe.txt 파일에 'café' 문자열을 쓴다.

4

In [2]:
open('cafe.txt').read()                                     # bytes로 저장된 파일을 str로 읽는다.

'café'

In [3]:
f = open('cafe.txt', 'w', encoding='utf_8')                  # cafe.txt 파일을 쓰기 모드로 열고, 인코딩은 utf_8로 한다.
f.write('café')                                             # 'café' 문자열을 쓴다.
f.close()                                                   # 파일을 닫는다.

In [4]:
import os

os.stat('cafe.txt').st_size                                 # 파일 크기를 알아낸다.

5

In [5]:
cafe_e = 'café'.encode()
print(cafe_e)
print(len(cafe_e))

cafe_d = b'caf\xc3\xa9'.decode()
print(cafe_d)
print(len(cafe_d))

b'caf\xc3\xa9'
5
café
4


In [6]:
# Mac OS에서는 기본 인코딩으로 UTF-8 을 사용하고 있다.
fp2 = open('cafe.txt')
print(fp2)
print(fp2.read())

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='UTF-8'>
café


In [7]:
# Windows 1252 인코딩 (cp1252)
f3 = open('cafe.txt', encoding='cp1252')
print(f3)
print(f3.read())

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp1252'>
cafÃ©


In [8]:
f4 = open('cafe.txt', 'rb')                       # 이진 모드로 읽는다.
print(f4)
print(f4.read())

<_io.BufferedReader name='cafe.txt'>
b'caf\xc3\xa9'


### 4. 5. 1 기본 인코딩 설정: 정신 나간 거 아냐?

In [9]:
# 시스템 기본 인코딩 알아보기
import sys
import locale

expression = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expression.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))


 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


locale.getpreferredencoding() 설정이 가장 중요하다. 이 함수를 통해 기본 텍스트 파일을 열고, 표준 입출력(sys.stdout, sys.stderr)을 리다이렉션할 때도 사용된다.

> **locale.getpreferrendencoding(do_setlocale=True)**\
사용자 환경에 따라 텍스트 데이터에 사용되는 인코딩을 반환한다. 이 함수가 반환하는 값은 추정치이다.

즉! '기본 인코딩에 의존하지 않는 것'이 가장 좋다!
유니코드 샌드위치 모델을 따르고 프로그램 내에서 항상 인코딩을 명시하면 많은 문제를 피할 수 있다.

## 4. 6 제대로 비교하기 위해 유니코드 정규화하기
유니코드에는 결합 문자가 있기 때문에 문자열 비교가 간단하지 않다. 앞 문자에 연결되는 발음 구별 기호(diacritical mark)는 인쇄할 때 앞 문자와 결합되어 출력된다.

'café'라는 단어를 예로 들면 cafe의 e에 액센트 기호가 붙은 것이다.
따라 해당 문자열은 'café'와 'cafe\u0301' 두 가지 방식으로 표현할 수 있다.


In [10]:
s1 = 'café'
s2 = 'cafe\u0301'

print(s1, s2)
print(len(s1), len(s2))
print(s1 == s2)

café café
4 5
False


In [11]:
print('e'.encode(encoding='utf_8'))

print('é́́'.encode(encoding='utf_8'))

b'e'
b'e\xcc\x81\xcc\x81\xcc\x81'


코드 포인트 `U+0301`은 *COMBINING CUTE ACCENT*다. 'e'에 붙어서 'é'를 만든다. 

e +  ́ -> é é́ é́́ é́́́ é́́́́

유니코드 표준에서는 따라서 'é'와 'e\u0301'은 규범적으로 동일하다고 하며, 애플리케이션은 두 시퀀스를 동일하게 처리해야 한다. 하지만 파이썬은 두 개의 코드 포인트 시퀀스를 보고 동일하지 않다고 판단한다.

이 문제를 해결하려면 Unicodedata.normalize() 함수가 제공하는 유니코드 정규화를 이용해야 한다. 이 함수의 첫 번째 인수는 'NFC', 'NFD', 'NFKC', 'NFKD' 네 가지 중 하나다. 

- NFC: 코드 포인트를 조합해서 짧은 문자열 생성
- NFD: 기본 문자와 결합 문자로 분리한다.

**이중 'NFC'는 W3C가 추천하는 정규화 형식**이므로 안전하게 `normalize('NFC', user_text)`하는 것이 권장된다.

(전기 저항을 나타내는 옴기호는 그리스어 대문자 오메가로 정규화된다. 겉모습은 똑같지만 다르다고 판단되므로 정규화해서 뜻하지 않은 문제를 예방해야 한다.)


[Unicode-table](https://unicode-table.com/kr/0301/)

In [12]:
from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301'

print(len(s1), len(s2))
print(len(normalize('NFC', s1)), len(normalize('NFC', s2)))
print(len(normalize('NFD', s1)), len(normalize('NFD', s2)))


4 5
4 4
5 5


In [13]:
from unicodedata import normalize, name

ohm = '\u2126'
ohm_c = normalize('NFC', ohm)

print("ohm:".rjust(25), name(ohm))
print("norm_ohm:".rjust(25), name(ohm_c))
print("ohm == ohm_c:".rjust(25), ohm == ohm_c)
print("norm_ohm == norm_hom_c:".rjust(25), normalize('NFC', ohm) == normalize('NFC', ohm_c))

                     ohm: OHM SIGN
                norm_ohm: GREEK CAPITAL LETTER OMEGA
            ohm == ohm_c: False
  norm_ohm == norm_hom_c: True


NFKC의 K는 호환성(Compatibility)을 나타낸다.

In [14]:
# NFKC
half = '½'
print(normalize('NFKC', half))
print(normalize('NFKD', half))

print(len(normalize('NFKC', half)), len(normalize('NFKD', half)))

four_squared = '4²'
print(normalize('NFKC', four_squared))
print(normalize('NFKD', four_squared))
print(len(normalize('NFKC', four_squared)), len(normalize('NFKD', four_squared)))

1⁄2
1⁄2
3 3
42
42
2 2


In [15]:
# NFKD
micro = 'µ'
micro_kc = normalize('NFKC', micro)

print(micro, micro_kc)
print(ord(micro), ord(micro_kc))        # ord() : 문자의 유니코드 코드 포인트를 반환
print(name(micro), name(micro_kc), sep=", ")

µ μ
181 956
MICRO SIGN, GREEK SMALL LETTER MU


NFKC와 NFKD normalization은 저장할 때에는 사용하지 않는다. 

다만, 검색이나 색인 생성 등의 작업을 할 때 유용하다.

1/2를 검색해서 ½ 를 찾을 수 있다면 사용자가 정말 기쁘겠죠?!


### 4. 6. 1 케이스 폴딩
case folding은 모든 텍스트를 소문자로 변환하는 연산이며, 약간의 변환을 동반한다. 케이스 폴딩은 `str.casefold()` 메서드를 통해 수행할 수 있다.

latin1 문자만 담고 있는 문자열인 경우 s.casefold()와 s.lower()는 같은 결과를 반환한다. 하지만 유니코드 문자열에 대해서는 다르다.
micro 기호(µ)는 동일해 보이지만 그리스 문자 μ로 변환되며, 샤프에스라고 불리는 독일어 에스체트 기호(ẞ)는 ss로 변환된다.

In [16]:
micro = 'µ'
micro_low = micro.lower()
micro_cf = micro.casefold()

print(name(micro))
print(name(micro_low))          # low() 를 통해서는 변환되지 않음 
print(name(micro_cf))
print(micro, micro_cf, micro_low)

MICRO SIGN
MICRO SIGN
GREEK SMALL LETTER MU
µ μ µ


micro 기호는 lower()를 통해 변환되지 않고 casefold()를 통해 그리스어 mu의 소문자로 변환된다. 
lower()와 casefold()가 서로 다른 문자를 반환하는 코드 포인트는 python 3.4 기준 116개라고 한다.

In [17]:
eszett = 'ß'
eszett_cf = eszett.casefold()

print(name(eszett))
print(name(eszett_cf[0]))
print(eszett, eszett_cf)

LATIN SMALL LETTER SHARP S
LATIN SMALL LETTER S
ß ss


Q? micro 기호는 normalize와 casefold한 결과가 같다. 그렇다면 왜 casefold를 사용해야 할까? casefold도 정규화의 일종이라고 할 수 있을까?

In [18]:
print(name(ohm))
print(name(ohm.lower()))

print(ohm.lower())
print(ohm.casefold())

print(name(normalize('NFC', ohm)))
print(name(ohm.casefold()))         # 정규화된 ohm의 소문자를 반환해줌

OHM SIGN
GREEK SMALL LETTER OMEGA
ω
ω
GREEK CAPITAL LETTER OMEGA
GREEK SMALL LETTER OMEGA


### 4. 6. 2 정규화된 텍스트 매칭을 위한 유틸리티 함수
최종적으로 정리하자면 NFC는 대부분의 애플리케이션에서 사용할 수 있는 최적의 정규화된 형태이고, str.casefold()는 대소문자를 구분 없이 문자를 비교할 때 가장 좋은 방법이다.

In [19]:
from normep import nfc_equal, fold_equal

s1 = 'café'
s2 = 'cafe\u0301'

print(s1 == s2)
print(nfc_equal(s1, s2))
print(fold_equal(s1, s2))

False
True
True


In [20]:
s3 = 'Straße'
s4 = 'strasse'

print(s3 == s4)
print(nfc_equal(s3, s4))
print(fold_equal(s3, s4))

False
False
True


### 4. 6. 3 극단적인 '정규화': 발음 구별 기호 제거하기
발음 구별 기호를 제고하는 것은 오탐이 발생할 수 있지만 너무 연연하지 말자!
액센트나 갈고리형 기호 등의 발음 기호를 제거하면 가독성이 높아진다.

`https://en.wikipedia.org/wiki/S%C3%A3o_Paulo` 를 `https://en.wikipedia.org/wiki/Sao_Paulo` 로 변환하는 것이다.

In [21]:
import unicodedata

def shave_marks(txt):
    """발음 구별 기호를 모두 제거한다."""
    
    # NFD 정규화로 문자와 기호를 분해
    norm_txt = normalize('NFD', txt)   
    
    # unicodedata.combining() 함수로 문자의 결합형 분류값을 구하고, 결합형 분류값이 0인 문자만 남긴다.
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return normalize('NFC', shaved)


order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
order = 'açaí'
print(shave_marks(order))

acai


In [22]:
import unicodedata

from torch import greater_equal

def shave_marks(txt):
    """발음 구별 기호를 모두 제거한다."""
    
    # NFD 정규화로 문자와 기호를 분해
    norm_txt = normalize('NFD', txt)   
    
    # unicodedata.combining(): 결합문자열을 정수로 반환. 문자열이 결합문자가 아니면 0을 반환
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))

    return normalize('NFC', shaved)


order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
greek = 'Ζέφυρος, Zéfiro'


print(shave_marks(order))
print(shave_marks(greek))

“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
Ζεφυρος, Zefiro


`έ`는 어차피 아스키코드에도 없는 문잔데, 극단적으로 액센트를 적용할 필요 없다! 아스키코드에 있는 것은 살려두자!

In [23]:
import string

def shave_marks_latin(txt):
    """라틴 기반 문자에서 발음 구별 기호를 모두 제거한다."""

    norm_txt = unicodedata.normalize('NFD', txt)
    latin_balse = False
    keepers = []

    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue    
        keepers.append(c)
        
        # c가 라틴 문자인지 확인
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)



order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
greek = 'Ζέφυρος, Zéfiro'

print(shave_marks_latin(order))
print(shave_marks_latin(greek))

“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
Ζέφυρος, Zefiro


In [24]:
# https://github.com/fluentpython/example-code/blob/master/04-text-byte/sanitize.py

single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     
    no_marks = no_marks.replace('ß', 'ss')          
    return unicodedata.normalize('NFKC', no_marks) 


order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'

print(dewinize(order))
print(asciize(order))

"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."
"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."


이렇게 텍스트 정규화를 해줄 수 있는 코드를 만나보았습니다.
언젠가 유용하게 쓸 수 있는 날을 기대해봅시다...!